In [1]:
# NLP를 이용한 Zero-shot Classification 분류 예시
# => 각 문장과 labels를 가지고, 문장(전제)과 labels(가설)간 관계가 최대 참(entailment) 일 확률을 구하는 방식

import torch
import numpy as np
from tqdm.notebook import tqdm
from transformers import BertTokenizerFast, BertForSequenceClassification

import sys
sys.path.append("..")
from myutils import seed_everything, GPU_info, pytorch_cos_sim

logfilepath:bwdataset_2022-03-07.log


In [2]:
model_path = '../model/classification/bmc_fpt_kowiki20200920.train_model_0225-ft-klue-nli-0303' 
vocab_path = "../model/classification/bmc_fpt_kowiki20200920.train_model_0225-ft-klue-nli-0303/vocab"

device = GPU_info()
print(device)

#seed 설정
seed_everything(111)

True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30
cuda:0


In [3]:
# tokenize 설정
tokenizer = BertTokenizerFast.from_pretrained(vocab_path)

In [4]:
# model 불러옴
model = BertForSequenceClassification.from_pretrained(model_path, output_hidden_states=True, num_labels=3)
model.eval()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(143772, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [5]:
print('prameters:{}'.format(model.num_parameters()))

prameters:196460547


In [28]:

'''
input_texts = ['오늘은 오후 부터 춥고 비가 올것 같다.']
labels = [
        ['오늘은 가끔 흐리고, 눈이 올수 있다'],
        ['여기 식당은 파스타가 맛있다'],
        ['오늘 증시는 내림으로 마감 하였다'],
        ['내일은 오전에는 흐리지만, 오후에는 날씨가 좋겠다']
]
'''

'''
#input_texts = ['오늘은 오전에는 흐리고 오후부터는 가끔씩 비가 오겠다']
input_texts = ['즐겨볼만한 스포츠']
labels = [['축구'],['야구'],['등산'],['낚시'],['날씨'],['상품권'],['교육']]
'''

input_texts = ['서울은 대한민국에 수도이며, 정치 경제 중심지이다',
          '내년 경제 성장은 4%대 성장을 이룰거라 예상된다',
          '프랑스 파리는 전세계 관광객들이 매년 찾는 관광도시이다',
          '올해에는 대통령 선거와 지방선거가 동시에 열린다',
          '오늘 날씨는 비가 내리고 매우 춥다',
          '손홍민이 영국 프리미어 축구 경기에서 11번째 골을 넣었다',
          '건조한 날씨에 산불을 조심해야 한다',
          '윈도우11 OS에 검색 기능을 강화 하였다',
          '한국은행은 올해 하반기 금리를 동결했다']
          

labels = [
        '여행',
         '투표',
         '증권',
         'IT',
         '날씨',
         '스포츠',
         '경제',
         '정치',
         '야구',
         '문화',
         '제주도'
         ]

# labels이 문장이고, input_texts가 keyword(단어)인 경우에는 True로 해줌
reverse_tokenizer = False


In [29]:
print(len(labels))
print(labels[1])
print(len(input_texts))
print(input_texts[1])

11
투표
9
내년 경제 성장은 4%대 성장을 이룰거라 예상된다


In [30]:
# NLP 문장을 테스트 : [CLS]input_text[SEP]labels[SEP] 형식으로 tokenizer 시킴 
tokenized_input = tokenizer([input_texts[0]], [labels[0]], return_tensors="pt")
token_str = [[tokenizer.convert_ids_to_tokens(s) for s in tokenized_input['input_ids'].tolist()[0]]]
print(token_str)
print(tokenized_input)

[['[CLS]', '서울', '##은', '대한민국', '##에', '수도', '##이며', ',', '정치', '경제', '중심지', '##이다', '[SEP]', '여행', '[SEP]']]
{'input_ids': tensor([[   101,  48253,  10892,  26168,  10530,  69283,  33542,    117, 119622,
         119641, 122451,  11925,    102, 120337,    102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [32]:

# 분류 문장을 불러와서, tokenize 한 다음, 모델에 넣고 출력값을 얻어옴
out_dict = {}
top_k = 5

for i, input_text in enumerate(input_texts):
    for idx, label in tqdm(enumerate(labels)):
        if reverse_tokenizer == True:
            tokenized_input = tokenizer(label, input_text, return_tensors="pt")  #역으로 표현
        else:
            tokenized_input = tokenizer(input_text, label, return_tensors="pt")
       
        token_str = [[tokenizer.convert_ids_to_tokens(s) for s in tokenized_input['input_ids'].tolist()[0]]]
        #print(token_str)
        #print(tokenized_input)

        outputs = model(**tokenized_input)
        #print(outputs)
        logits = outputs.logits
        #print(logits)

        '''
        # ouput_hidden_states = True일때 출력되는 hidden_state 값을 가지고 임베딩 구할수 있음
        hidden_states = outputs.hidden_states
        layer_idx = 0
        batch_idx = 0
        token_idx = 0
        print('hidden_states')
        print("-레이어 수:{}".format(len(hidden_states)))
        print("-배치 수: {}".format(len(hidden_states[layer_idx])))
        print("-토큰 수 : {}".format(len(hidden_states[layer_idx][batch_idx])))
        print("-hidden 유닛 수 : {}".format(len(hidden_states[layer_idx][batch_idx][token_idx])))
        '''

        # logits 에 softmax 를 취해서 총합이 1이되는 확률 분포로 만듬
        prob = logits.softmax(dim=1)
        #print(prob)

        # entailment(참) 일 확률 중에서 가장  높은 거 선택 
        entailment_prob = prob[0][0]      # 참(수반)일 경우 확률
        #contradiction_prob = prob[0][1]  #거짓(모순)일 경우 확률 
        #netral_prob = prob[0][2]         # 중립일 경우 확률
        #print(f'input:{input_text[0]}, label:{label[0]}, 참일 확률:{entailment_prob}')
        
              
        # 사전 key로 순번으로 하고, entailment(참) 일 확률를 저장함
        key = str(idx+1)
        out_dict[key] = entailment_prob
      
        '''      
        # 확률을 소수점 2자리에서 반올림
        entailment_prob = round(prob[0][0].item(),2) # 참(수반)일 경우 확률
        contradiction_prob = round(prob[0][1].item(),2) # 거짓(모순)일 경우 확률
        netral_prob = round(prob[0][2].item(),2) # 중립일 경우 확률

        print(f'input:{input_text}, label:{label}')
        print(f'참:{entailment_prob}, 거짓:{contradiction_prob}, 중립:{netral_prob}')

        if torch.argmax(prob) == 0:
            pred = "참(entailment)"
        elif torch.argmax(prob) == 1:
            pred = "거짓(contradiction)"
        else: 
            pred = "중립(netral)"

        print(f'{pred}')
        '''      

    # 사전 정렬(value(유사도)로 reverse=True 하여 내림차순으로 정렬함)
    sorted_dict = sorted(out_dict.items(), key = lambda item: item[1], reverse = True)
    #print(sorted_dict)

    print(f'{input_text}\n')

    # 내립차순으로 정렬된 사전출력 
    for count in (sorted_dict):
        value = count[1].tolist() # count[1]은 2차원 tensor이므로 이를 list로 변환
        #print(value)
        idx = int(count[0])
        #print(idx)
        #print(labels[idx-1][0])
        print(labels[idx-1].strip(), "(Score: %.6f)" % (value))

0it [00:00, ?it/s]

서울은 대한민국에 수도이며, 정치 경제 중심지이다

경제 (Score: 0.997608)
정치 (Score: 0.991747)
제주도 (Score: 0.000111)
문화 (Score: 0.000086)
IT (Score: 0.000068)
야구 (Score: 0.000065)
증권 (Score: 0.000052)
투표 (Score: 0.000050)
스포츠 (Score: 0.000050)
여행 (Score: 0.000050)
날씨 (Score: 0.000050)


0it [00:00, ?it/s]

내년 경제 성장은 4%대 성장을 이룰거라 예상된다

경제 (Score: 0.324677)
정치 (Score: 0.000075)
투표 (Score: 0.000059)
야구 (Score: 0.000056)
날씨 (Score: 0.000056)
문화 (Score: 0.000053)
여행 (Score: 0.000052)
스포츠 (Score: 0.000048)
증권 (Score: 0.000047)
제주도 (Score: 0.000047)
IT (Score: 0.000042)


0it [00:00, ?it/s]

...

건조한 날씨에 산불을 조심해야 한다

날씨 (Score: 0.999679)
여행 (Score: 0.000067)
야구 (Score: 0.000053)
제주도 (Score: 0.000052)
스포츠 (Score: 0.000051)
증권 (Score: 0.000048)
경제 (Score: 0.000048)
정치 (Score: 0.000048)
IT (Score: 0.000048)
문화 (Score: 0.000046)
투표 (Score: 0.000040)


0it [00:00, ?it/s]

윈도우11 OS에 검색 기능을 강화 하였다

IT (Score: 0.000084)
여행 (Score: 0.000055)
정치 (Score: 0.000053)
스포츠 (Score: 0.000052)
문화 (Score: 0.000051)
야구 (Score: 0.000047)
날씨 (Score: 0.000044)
증권 (Score: 0.000043)
제주도 (Score: 0.000041)
투표 (Score: 0.000040)
경제 (Score: 0.000036)


0it [00:00, ?it/s]

한국은행은 올해 하반기 금리를 동결했다

경제 (Score: 0.002114)
증권 (Score: 0.000109)
정치 (Score: 0.000077)
스포츠 (Score: 0.000059)
문화 (Score: 0.000055)
여행 (Score: 0.000054)
투표 (Score: 0.000053)
야구 (Score: 0.000051)
IT (Score: 0.000046)
날씨 (Score: 0.000043)
제주도 (Score: 0.000041)
